# Model training

## Import all the required packages

In [1]:
import os
dir_path = os.path.dirname(os.getcwd())

In [2]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import sys  
sys.path.append(os.path.join(dir_path, "src"))
from word_embeddings import get_embeddings
from clean_comments import clean
from processing import process_txt
from train import train_model
from sklearn import feature_extraction, preprocessing, naive_bayes
import pickle

## Load Processed Dataset

In [3]:
processed_data = os.path.join(dir_path, 'data', 'processed', 'processed_stem_data.csv')

In [4]:
df = pd.read_csv(processed_data)

In [5]:
df.head()

,Unnamed: 0,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0,0000997932d777bf,0,0,0,0,0,0,explan edit made usernam hardcor metallica fan...
1,1,000103f0d9cfb60f,0,0,0,0,0,0,aww match background colour seemingli stuck th...
2,2,000113f07ec002fd,0,0,0,0,0,0,hey man realli tri edit war guy constantli rem...
3,3,0001b41b1c6bb37e,0,0,0,0,0,0,ca make real suggest improv wonder section sta...
4,4,0001d958c54c6e35,0,0,0,0,0,0,sir hero chanc rememb page


In [6]:
### fill NA for any missing data 
df['comment_text'].fillna("missing", inplace=True)

In [7]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
corpus = df['comment_text']

## Convert Text Comments into Vectors using Bag-of-Words or TF-IDF 

We know that machine learning models doesn't accept input in the text format. So we need to convert the text data into vector form, it is also called **Word Embeddings**. Word Embeddings can be broadly classified as:
1. Frequency based - Most popular techniques are **Bag-of-Words**, **TF-IDF**
2. Prediction based - Most popular techniques are **Word2vec** and **Glove**



Here we will be using **Bag-of-Words** and **TF-IDF**<br>
<br>**Bag-of-Words(BOW)** - To get the embeddings with BOW technique, firstly we will have to make a dictionary with keys as a words from the test data and count of each word occurance as a value, then sort the dictionary in descending order of its values.Then these words from the dictionary are the names of our independed features and we can also choose how many features we want to use by selecting the top n words from the dictionary. Now these features will have values 0 or 1 based on if the word exists in the sentence.
<br>
**Disadvantage** of BOW - Word Embeddings have either 0's and 1's as a values, no weights are given to the words according to their importance in the sentence. That means we can not get the sementics of the sentence.

**TF-IDF** - It stands for Term Frequency - Inverse Document Frequency
<br> To get embedding with TF-IDF, we calculate Term frequency and Inverse Document Frequency seperate and then multiply them together to get TF-IDF.
Formulas to calculate TF-IDF: 
<br>
<br>**TF :**  $$\frac{Number\, of\, repetition\, of\, word\, in\, a\, sentence}{Number\, of\, words\, in\, a\, sentence}$$ 
**IDF :**$$log\Bigg[\frac{Total\, Number\, of\,sentences}{Number\, of\, sentences\, containing\, the \, word}\Bigg]$$ 
<br>
**TF-IDF :** $TF * IDF$ $$\Bigg[\frac{Number\, of\, repetition\, of\, word\, in\, a\, sentence}{Number\, of\, words\, in\, a\, sentence}\Bigg]*log\Bigg[\frac{Total\, Number\, of\,sentences}{Number\, of\, sentences\, containing\, the \, word}\Bigg]$$ 
<br>In **TF-IDF** also, we create dictionary of words with their count of occurance. **TF** assign more weightage to the word which repeat multiple times in the sentance where as **IDF** decreases the weightage to word as number of sentences containing the increases. Here, feature vectors not only contains 0's and 1' but does contain other values depending on the significance of the word in the sentence. This technique retains the sementics of the sentence to some extent so it should perform better than BOW.
<br>Here **TF-IDG** can have zero value for the word which existed in every sentence and give more weightage to least occured words that means it could cause over-fitting problem but that is yet to discove. 

We will try Bag-of-Words and TF-IDFto get our features for `X_train`  and `X_test` data. The resultant embeddings are in numpy array format, if we have a look at the the embeddings we will know it is high dimensional sparse data.

## Matrix used to evaluate the models

Multi-label classification problems must be assessed using different performance measures than single-label classification problems.
<br>
<br>
Jaccard similarity, or the Jaccard index, is the size of the intersection of the predicted labels and the true labels divided by the size of the union of the predicted and true labels. It ranges from 0 to 1, and 1 is the perfect score.Here we are taking mean so 100 is perfect score. This function can be imported from *Sklearn.metrics* but just to have better usderstanding we are defining the `j_score` function.
<br>
<br>
We will also look at *F1-score* and *ROC score*

## Model Training

### Models we are using for text classification
1. Logistics Regression
2. Naive Bayes (NB)


Which model to use?
<br>
Which is the fastest model for high dimensional sparse data?  - **Logistic regression**
<br>We will use Logistic regression for this dataset to start with and the solver we are using is 'sag' as it is faster for large datsets.


### Logistic Regression

Logistic regression is similar to `Linear Regression` except it predicts whether something is **True** or **False**, instead of predicting a continuous value. Also instead of fitting a line to the data it fits an **"S"** shaped "logistic function" which is called `sigmoid function`.
<br> In `Logistic Regression ` we don't need to do much to classify an instance. All we have to do is calculate the sigmoid of the vectorunder test multiplied by the weights optimized. If sigmoid gives a value greater than 0.5 the class is 1 and it is 0 otherwise.


### Initiate models

In [12]:
logreg = LogisticRegression(solver='sag')

In [56]:
lr_clf, lr_vectorizer, lr_sumry = train_model(logreg, corpus, df[labels])

... Performing train test split
... Extracting features
... Training LogisticRegression model


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 2.4208497375328086
F1 Score : 0.05396290560876806
None
Accuracy is 0.8986538991803074
ROC_AUC - 0.514002011361763
check model accuracy on input_string [[0 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36071     7]
  [ 3620   195]]

 [[39482     5]
  [  390    16]]

 [[37741     9]
  [ 2053    90]]

 [[39788     0]
  [  104     1]]

 [[37873     9]
  [ 1974    37]]

 [[39535     1]
  [  354     3]]]


### Logistic Regression Results

In [58]:
summary_lr = pd.DataFrame(lr_sumry, index = ['accuracy', 'jaccard score', 'F1_score', 'roc_score'], 
                          columns= [lr_clf.estimators_[0]])
summary_lr

,LogisticRegression(solver='sag')
accuracy,0.898654
jaccard score,2.420850
F1_score,0.053963
roc_score,0.514002


<br>
### Naive Bayes

It is a classifier under supervised ML group based on probalistic logic. Probabilistic logic it uses is  `Bayes Theorem`  which gives probability of an event based on prior knowledge of condition that might be related to event
<br>
<br>
**P(Class|Features) :**  $$\frac{P(Features|Class) * P(Class)}{P(Features)}$$ 
It assumes that the probability of a one word doesn't depends on any other word in the document. We know this is unrealistic. That's why it is known as `naive Bayes`. Despite of its incorrect assumptions, `naive Bayes` is effective at classification. 
<br>Moreover, assuming conditional independence among the features in the dataset, it reduces the need of large training data.
#### Multinomial NB
In our use case we will be using this `Multinomial NB` because it assumes count data which means each feature represents an integer count of some-thing, in our problem it is that how often a word appears in a sentence.

### Initiate models

In [8]:
n_bayes = naive_bayes.MultinomialNB()

In [43]:
nb_clf, nb_vectorizer, nb_sumry = train_model(n_bayes, corpus, df[labels] )

... Performing train test split
... Extracting features
... Training MultinomialNB model
... Computing accuracy
... Saving model in model directory


Model evaluation
------
Clf:  MultinomialNB
Jaccard score: 34.340896239751125
F1 Score : 0.49689026569064043
None
Accuracy is 0.8983029604191212
ROC_AUC - 0.786800037695929
------
Multilabel confusion matrix 
 [[[35348   730]
  [ 1474  2341]]

 [[38980   507]
  [  130   276]]

 [[37240   510]
  [  673  1470]]

 [[39444   344]
  [   52    53]]

 [[37163   719]
  [  810  1201]]

 [[38993   543]
  [  197   160]]]


### Naive Bayes Results

In [44]:
summary_nb = pd.DataFrame(nb_sumry, index = ['accuracy', 'jaccard score', 'F1_score', 'roc_score'],
                          columns= [nb_clf.estimators_[0]])
summary_nb

,MultinomialNB()
accuracy,0.898303
jaccard score,34.340896
F1_score,0.496890
roc_score,0.786800


<br>
**Naive Bayes or Logistic regression !!**
<br>Compairing the confusion matrixs and Jaccard score, Naive Bayes clearly out performed Linear regression and Naive Bayes even tends to get trained faster.
Just yet we can't decide, we need to try different model evaluation techniqes first.

### K FOLD Cross Validation

#### Logistic Regression - K Fold cross validation with Gridsearch

We will use GridSearchCV to evaluate the model using different values of **C**

In [9]:
param_grid = {"estimator__C": [0.001, 0.01, 0.1, 1, 10, 100]}

In [10]:
#Train-test split
print("... Performing train test split")
X_train, X_test, y_train, y_test = model_selection.train_test_split(corpus,df[labels],
                                                                    test_size=0.25,random_state=42)

... Performing train test split


In [11]:
## Features extraction with word embedding
print("... Extracting features")
Xv_train, Xv_test, vectorizer = get_embeddings(X_train, X_test,
                                                          max_feature = 1000 , embedding_type= 'bow')

... Extracting features


In [14]:
# Uncomment to reproduce results
clf_lr = OneVsRestClassifier(logreg)

In [15]:
# Uncomment to reproduce results
gs_lr = GridSearchCV(clf_lr, param_grid ,scoring = 'f1_micro', cv=3)

In [25]:
# Uncomment to run GridSerch with K-fold cross validation
gs_lr.fit(Xv_train, y_train)

In [26]:
print("Accuracy score of the best estimator : {}".format(gs_lr.score(Xv_test, y_test )))

Accuracy score of the best estimator : 0.0742671009771987


In [28]:
print("Best estimator is : {}".format(gs_lr.best_estimator_))

Best estimator is : OneVsRestClassifier(estimator=LogisticRegression(C=0.1, solver='sag'))


<br>
As we can see that *Logistic Regression* performed well with **C = 0.1** on 3 Folds of cross validation. We will use this value of **C** with the various combination of *maximum_features* and *embedding types* in next part to check the performance of *Logistic Regression* during the best model selection process

<br>
#### Naive Bayes - K-fold cross validation

In [68]:
### Naive bayes
nbayes = OneVsRestClassifier(naive_bayes.MultinomialNB())
nbayes.fit(Xv_train, y_train)
cv_score= cross_val_score(nbayes,Xv_train,y_train,cv=3)
print('Naive Bayes Cross Validation score = {}'.format(cv_score))

Naive Bayes Cross Validation score = [0.8985787  0.89827789 0.89804973]


<br>
We can see even with the K-fold cross validation *Naive Bayes* is giving consistant results for the accuracy

## Best Model Selection

In [33]:
logreg = LogisticRegression(solver='sag', C= 0.1)
classifier = [logreg, n_bayes]
max_feature = [1200, 1500, 1700, 2000]
embedding= ['bow', 'tfidf']

#### Training models with all different combinations of the parameters stated in above cell

In [73]:
summary_report =  []
combinations =[]
best_jscore = 1
for estimator in classifier:
  print(".... Processing {}".format(estimator))
  for m in max_feature:
    for n in embedding:
      print(".... Combination {}, {} ".format(m,n))
      combinations.append((estimator.__class__.__name__, m, n))
      clf, vectorizer, score_sumry = train_model(estimator, corpus, df[labels], max_feature=m, embedding= n )
      summary_report.append(score_sumry)
      if score_sumry[1] > best_jscore:
        best_jscore =score_sumry[1]
        best_param = {"classifier": clf.__class__.__name__, "max_features": m, "embedding_type":n}
      
      
print(summary_report)
print(best_param)      


.... Processing LogisticRegression(C=0.1, solver='sag')
.... Combination 1200, bow 
... Performing train test split
... Extracting features
... Training LogisticRegression model


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 2.4405347769028873
F1 Score : 0.05424650879849518
None
Accuracy is 0.8986538991803074
ROC_AUC - 0.5140798836765966
check model accuracy on input_string [[0 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36071     7]
  [ 3620   195]]

 [[39482     5]
  [  390    16]]

 [[37742     8]
  [ 2051    92]]

 [[39788     0]
  [  104     1]]

 [[37873     9]
  [ 1974    37]]

 [[39534     2]
  [  354     3]]]
.... Combination 1200, tfidf 
... Performing train test split
... Extracting features
... Training LogisticRegression model
...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 30.15339663988314
F1 Score : 0.3453158479919651
None
Accuracy is 0.9127165166821247
ROC_AUC - 0.6244354518201726
check model accuracy on input_string [[1 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36007

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 2.5090223097112863
F1 Score : 0.05499686661789316
None
Accuracy is 0.8987040332890482
ROC_AUC - 0.5142739489194659
check model accuracy on input_string [[0 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36071     7]
  [ 3617   198]]

 [[39483     4]
  [  390    16]]

 [[37742     8]
  [ 2051    92]]

 [[39788     0]
  [  104     1]]

 [[37873     9]
  [ 1971    40]]

 [[39535     1]
  [  354     3]]]
.... Combination 1500, tfidf 
... Performing train test split
... Extracting features
... Training LogisticRegression model
...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 30.103963612735544
F1 Score : 0.34236089482335696
None
Accuracy is 0.9126663825733838
ROC_AUC - 0.6232057010257515
check model accuracy on input_string [[1 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[360

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 2.523786089238846
F1 Score : 0.055999042728626915
None
Accuracy is 0.8987291003434187
ROC_AUC - 0.5145313270069294
check model accuracy on input_string [[0 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36071     7]
  [ 3616   199]]

 [[39483     4]
  [  390    16]]

 [[37742     8]
  [ 2051    92]]

 [[39788     0]
  [  104     1]]

 [[37873     9]
  [ 1971    40]]

 [[39536     0]
  [  353     4]]]
.... Combination 1700, tfidf 
... Performing train test split
... Extracting features
... Training LogisticRegression model
...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 29.919413919413945
F1 Score : 0.33755166106456674
None
Accuracy is 0.9124658461384203
ROC_AUC - 0.6211262166735317
check model accuracy on input_string [[1 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[360

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarn

...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 2.5352690288713915
F1 Score : 0.05614149422955315
None
Accuracy is 0.8987291003434187
ROC_AUC - 0.5145702133033986
check model accuracy on input_string [[0 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[36071     7]
  [ 3616   199]]

 [[39483     4]
  [  390    16]]

 [[37742     8]
  [ 2050    93]]

 [[39788     0]
  [  104     1]]

 [[37873     9]
  [ 1971    40]]

 [[39536     0]
  [  353     4]]]
.... Combination 2000, tfidf 
... Performing train test split
... Extracting features
... Training LogisticRegression model
...Computing accuracy
...Saving model in model directory


Model evaluation
------
Clf:  LogisticRegression
Jaccard score: 29.527241632054736
F1 Score : 0.33246160601041497
None
Accuracy is 0.9124658461384203
ROC_AUC - 0.6187219763849033
check model accuracy on input_string [[1 0 0 0 0 0]]
None
------
Multilabel confusion matrix 
 [[[360

### Performance Summary of all the Models Trained on different sets of Parameters

In [74]:
summary_r = pd.DataFrame(summary_report, index = [combinations], columns= ['accuracy', 'jaccard score', 'F1_score', 'roc_score'],)
summary_r

,accuracy,jaccard score,F1_score,roc_score
"(LogisticRegression, 1200, bow)",0.898654,2.440535,0.054247,0.514080
"(LogisticRegression, 1200, tfidf)",0.912717,30.153397,0.345316,0.624435
"(LogisticRegression, 1500, bow)",0.898704,2.509022,0.054997,0.514274
"(LogisticRegression, 1500, tfidf)",0.912666,30.103964,0.342361,0.623206
"(LogisticRegression, 1700, bow)",0.898729,2.523786,0.055999,0.514531
"(LogisticRegression, 1700, tfidf)",0.912466,29.919414,0.337552,0.621126
"(LogisticRegression, 2000, bow)",0.898729,2.535269,0.056141,0.514570
"(LogisticRegression, 2000, tfidf)",0.912466,29.527242,0.332462,0.618722
"(MultinomialNB, 1200, bow)",0.898679,35.016246,0.497675,0.790709
"(MultinomialNB, 1200, tfidf)",0.913619,31.526869,0.413841,0.654783


In [75]:
summary_all_combinations = summary_r.sort_values(by=['jaccard score'],ascending=False)
summary_all_combinations

,accuracy,jaccard score,F1_score,roc_score
"(MultinomialNB, 2000, bow)",0.897651,36.211174,0.503719,0.807032
"(MultinomialNB, 1700, bow)",0.897701,36.000000,0.500272,0.801386
"(MultinomialNB, 1500, bow)",0.897350,35.429208,0.495729,0.795791
"(MultinomialNB, 1200, bow)",0.898679,35.016246,0.497675,0.790709
"(MultinomialNB, 2000, tfidf)",0.913669,33.379039,0.411415,0.655950
"(MultinomialNB, 1700, tfidf)",0.913519,32.990456,0.412711,0.655797
"(MultinomialNB, 1500, tfidf)",0.913794,32.495376,0.409450,0.654051
"(MultinomialNB, 1200, tfidf)",0.913619,31.526869,0.413841,0.654783
"(LogisticRegression, 1200, tfidf)",0.912717,30.153397,0.345316,0.624435
"(LogisticRegression, 1500, tfidf)",0.912666,30.103964,0.342361,0.623206


with **C =0.1** logistic regression is doing much worst that **C= 1**

Its evident from above results that *Logistic Regression* perform well with continuous data (when features were computed with "TF-IDF") and *Naive Bayes* do good when data is in discrete form (When features were computer with "Bag-of Words"). Considering *Jaccard score*, *F1-score*, and *ROC_AUC score* **Naive Bayes** perform well, though when we train *Logistic Regression* with **C=1**, it does perform better than *Naive Bayes* only in terms of *Jaccard score*. I am sure with further fine tuning of the hyper parameters, we could get good *Logistic Regression* Model but its is quite expensive to get there in terms of computations and time. So we will go with **Naive Bayes** using *"Bag-of-Words"* embedding technique with *max_features* count as 2000

## Training the best model

In [9]:
final_clf, final_vectorizer , model_summary = train_model(n_bayes, corpus, df[labels], max_feature=2000, embedding= "bow")

... Performing train test split
... Extracting features
... Training MultinomialNB model
... Computing accuracy


Model evaluation
------
Clf:  MultinomialNB
Jaccard score: 36.21117365019788
F1 Score : 0.503718602592237
None
Accuracy is 0.8976512170054897
ROC_AUC - 0.8070323089999197
------
Multilabel confusion matrix 
 [[[35269   809]
  [ 1290  2525]]

 [[38955   532]
  [  120   286]]

 [[37127   623]
  [  612  1531]]

 [[39371   417]
  [   49    56]]

 [[37065   817]
  [  703  1308]]

 [[38878   658]
  [  171   186]]]


### Best Model Results

In [46]:
summary_best_model = pd.DataFrame(model_summary, index = ['accuracy', 'jaccard score', 'F1_score', 'roc_score'],
                          columns= [final_clf.estimators_[0]])
summary_best_model

,MultinomialNB()
accuracy,0.897651
jaccard score,36.211174
F1_score,0.503719
roc_score,0.807032


### Save Model

In [48]:
## Save model
pkl_file = os.path.join(dir_path,'model', 'final_model.pkl')
file = open(pkl_file,"wb")
pickle.dump(final_clf,file)
file.close()
print("...Model saved in model directory")
    

...Model saved in model directory


In [25]:
## Save vectorizer
print("...Saving vectorizer in model directory")
pkl_file = os.path.join(dir_path,'model', 'final_vectorizer.pkl')
file = open(pkl_file,"wb")
pickle.dump(final_vectorizer,file)
file.close()
print("...Saved vectorizer in model directory")

...Saving vectorizer in model directory
...Saved vectorizer in model directory


### Ignore below part

<br>
Lets do testing on unseen data

In [52]:
input_str = ["that is so good, i am so happy bitch"]
input_str = clean(input_str[0])
input_str = process_txt(input_str, stemm= True)
input_str = nb_vectorizer.transform([input_str])
input_str


<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [53]:
### Open the saved mode.pkl
pkl_file = os.path.join(dir_path, 'model', 'MultinomialNB')
open_file = open(pkl_file, "rb")
model = pickle.load(open_file)
open_file.close()

In [54]:
model

OneVsRestClassifier(estimator=MultinomialNB())

In [55]:
model.predict(input_str)

array([[1, 0, 1, 0, 1, 0]])

# Check if it needs to stay here
**`Bag-of-Words` and `TF-IDF`**
<br>Accuracies for `Bag-of-Words` and `TF-IDF` are not same but their difference is also not very significant for both Linear regression and Naive Bayes 
<br>Accuracy remained same - `Identity hate`, `threat`
<br>Accuracy remained almost same - `Severe_toxic`
<br>Accuracy improved little bit  with the use of TF-IDF but not very significant change for `Toxic`, `Obscene`, `insult`.
One possible reason for not seeing the expected significant improvement with the use of `TF-IDF` could be - the human raters didn't care for the semantics of the sentances and rated the comment based on the presence of toxic words.
In this case, we will choose `Bag-of-Words` because the performance is almost same as `TF-IDF` but less chance of overfitting.


### Stratified K Fold Cross Validation

In [ ]:
Xc = corpus

**Logistic Regression**

In [ ]:
skf_lr_accuracy = []
for label in labels:
    print('\n... PROCESSING {}'.format(label.upper()))
    # train the model
    clf = logreg

    
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(Xc,df[label])
    
    score_lr_acc = []
    score_lr_jaccard = []
    score_lr_roc = []
    score_lr_f1 = []
    i = 1
    for train_index, test_index in skf.split(Xc,df[label]):
        Y = df[label]
        #print("Train:", train_index, "validation:", test_index)
        X1_train, x1_test = Xc.iloc[train_index], Xc.iloc[test_index]
        y1_train, y1_test = Y.iloc[train_index], Y.iloc[test_index]

        ### Convert into word embeddings
        train_feat, test_feat, vectorizer = get_embeddings(X1_train,x1_test)
#       bw_vectorizer = feature_extraction.text.CountVectorizer(max_features= 100)
#       X1_train = bw_vectorizer.fit_transform(X1_train).toarray()
#       x1_test = bw_vectorizer.fit_transform(x1_test).toarray()

        clf.fit(train_feat, y1_train)
        prediction_lr = clf.predict(test_feat)
        
        acc_lr_score = accuracy_score(y1_test,prediction_lr)
        score_lr_acc.append(acc_lr_score)
        
        jaccard_lr = j_score(pd.DataFrame(y1_test),pd.DataFrame(prediction_lr))
        score_lr_jaccard.append(jaccard_lr)
        
        roc_lr = roc_auc_score(y1_test,prediction_lr)
        score_lr_roc.append(roc_lr)
        
        print("----- Processed {} fold".format(i))
        print(print_score(prediction_lr, y1_test, clf))
        
        i = i+1
        
    print("Model evaluation")
    print("------")
    print("ROC_AUC - {}".format(score_lr_roc))
    print('Average accuracy for 5 Kfolds in {} category {}'.format(label, np.array(score_lr_acc).mean()))
    print("------")
    #print(type(score))
    skf_lr_accuracy.append(np.array(score_lr_acc).mean())
print("\n Mean accuracies of all six labeles: {}".format(skf_lr_accuracy))
    

<br>
**Naive Bayes**

In [ ]:
classifier = n_bayes
skf_nb_accuracy = []
for label in labels:
    print('\n... PROCESSING {}'.format(label.upper()))
    # train the model
    classifier = n_bayes

    
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(Xc,df[label])
    
    score_label = []
    score_nb_jaccard = []
    score_nb_roc = []
    score_nb_f1 = []
    i = 1
    for train_index, test_index in skf.split(Xc,df[label]):
        Y = df[label]
        #print("Train:", train_index, "validation:", test_index)
        X1_train, x1_test = Xc.iloc[train_index], Xc.iloc[test_index]
        y1_train, y1_test = Y.iloc[train_index], Y.iloc[test_index]

        ### Convert into word embeddings
        train_feat, test_feat, vectorizer = get_embeddings(X1_train,x1_test)
#       bw_vectorizer = feature_extraction.text.CountVectorizer(max_features= 100)
#       X1_train = bw_vectorizer.fit_transform(X1_train).toarray()
#       x1_test = bw_vectorizer.fit_transform(x1_test).toarray()

        classifier.fit(train_feat, y1_train)
        prediction = classifier.predict(test_feat)
        
        acc_score = accuracy_score(y1_test,prediction)
        score_label.append(acc_score)
        
        jaccard_nb = j_score(pd.DataFrame(y1_test),pd.DataFrame(prediction))
        score_nb_jaccard.append(jaccard_nb)
        
        roc_nb = roc_auc_score(y1_test,prediction)
        score_nb_roc.append(roc_nb)
        
        print("----- Processed {} fold".format(i))
        print(print_score(prediction, y1_test, classifier))
        
        i = i+1
        
    print("Model evaluation")
    print("------")
    print("ROC_AUC - {}".format(score_nb_roc))
    print('Average accuracy for 5 Kfolds in {} category {}'.format(label, np.array(score_label).mean()))
    print("------")
    #print(type(score))
    skf_nb_accuracy.append(np.array(score_label).mean())
print("\n Mean accuracies of all six labeles: {}".format(skf_nb_accuracy))
    

### Trying multiple values of C in Linear regression model using simple Train-Test split method

In [ ]:
best_accuracy = {'toxic': 0, 'severe_toxic': 0, 'obscene': 0, 'threat': 0, 'insult': 0, 'identity_hate': 0}
best_c_value = {}
for c in [0.001, 0.01, 0.1, 1, 10, 100]:
   ### Naive bayes
  print('\n... Processing  C= {}'.format(c)) 
  accuracy_lr = []
  for label in labels:
      print('... Processing {}'.format(label))
      # train the model 
      logreg = OneVsRestClassifier(LogisticRegression(C=c ,solver='sag'))
      logreg.fit(Xv_train, y_train[label])
      # compute the testing accuracy
      prediction = logreg.predict(Xv_test)
      score = (accuracy_score(y_test[label], prediction))
      accuracy_lr.append(score)
      if score > best_accuracy[label]:
        best_accuracy[label] = score
        best_c_value[label] = c
      print('Validation accuracy is {}'.format(accuracy_score(y_test[label], prediction)))
  print("Accuracy_lr: {}".format(accuracy_lr))
print("\n Best accuracy : {}".format(best_accuracy))
print("Best parameter : {}".format(best_c_value))

In [ ]:
# Output of above cell
# Best accuracy : {'toxic': 0.9107612864412303, 'severe_toxic': 0.9900233123605645, 'obscene': 0.9515955180106785, 'threat': 0.9973679592911037, 'insult': 0.9509688416514176, 'identity_hate': 0.9910259945353821}
# Best parameter : {'toxic': 0.1, 'severe_toxic': 0.001, 'obscene': 0.1, 'threat': 0.001, 'insult': 0.1, 'identity_hate': 0.001}


In [21]:
##### Old version to try Kfolf CV

# ### OneVsRestClassifier
# for classifier in [n_bayes,logreg]:
    
#     print('... Processing {}'.format(classifier))
    
#     #Train-test split
#     print("... Performing train test split")
#     X_train, X_test, y_train, y_test = model_selection.train_test_split(corpus,df[labels],
#                                                                     test_size=0.25,random_state=42)
    
#     ## Features extraction with word embedding
#     print("... Extracting features")
#     Xv_train, Xv_test, vectorizer = get_embeddings(X_train, X_test,
#                                                           max_feature = 1000 , embedding_type= 'bow')
    
#     # train the model 
#     print('... Training {} model'.format(classifier.__class__.__name__))
    
#     # train the model 
#     clf = OneVsRestClassifier(classifier)
#     clf.fit(Xv_train, y_train)

#     # compute the testing accuracy
#     prediction = clf.predict(Xv_test)
    
#     score = (accuracy_score(y_test, prediction))
#     cv_score= cross_val_score(classifier,corpus,df[labels],cv=5)
#     roc_lr = roc_auc_score(y1_test,prediction_lr)
    
#     ## Save model
#     pkl_file = os.path.join(dir_path,'model', classifier.__class__.__name__)
#     file = open(pkl_file,"wb")
#     pickle.dump(clf,file)
#     file.close()
    
#     #### Prediction on comment 
        
#     input_str = ["i'm going to kill you nigga, you are you sick or mad, i don't like you at all"]
#     input_str = clean(input_str[0])
#     input_str = process_txt(input_str, stemm= True)
#     input_str = vectorizer.transform([input_str])
    
#     print("KFold score {}".format(cv_score))
#     print(print_score(prediction, y_test, classifier))
#     print(print("check model accuracy on input_string {}".format(clf.predict(input_str))))
#     print('Validation accuracy is {}'.format(score))
#     print("------")
    


In [22]:
# ### Linear regression for kfold CV
# cv_accuracy_lr = []
# accuracy_lr = []
# for label in labels:
#     print('... Processing {}'.format(label))
#     # train the model 
#     clf = OneVsRestClassifier(logreg)
#     clf.fit(Xv_train, y_train[label])
#     # compute the testing accuracy
#     prediction = logreg.predict(Xv_test)
#     score = (accuracy_score(y_test[label], prediction))
#     cv_score= cross_val_score(logreg,X,df[labels],cv=10)
#     accuracy_lr.append(score)
    
#     print('Validation accuracy is {}'.format(accuracy_score(y_test[label], prediction)))
#     print('\n cv_score = {}'.format(cv_score))
# print("\n Accuracy_lr: {}".format(accuracy_lr))

In [ ]:
# ### Naive bayes
# accuracy_nb = []
# for label in labels:
#     print('\n... Processing {}'.format(label))
#     # train the model 
#     nbayes = OneVsRestClassifier(naive_bayes.MultinomialNB())
#     nbayes.fit(Xv_train, y_train[label])
#     # compute the testing accuracy
#     prediction = nbayes.predict(Xv_test)
#     score = (accuracy_score(y_test[label], prediction))
#     cv_score= cross_val_score(nbayes,X,df[labels],cv=10)
#     accuracy_nb.append(score)
#     print('Validation accuracy is {}'.format(score))
#     print('cv_score = {}'.format(cv_score))
# print("\n Accuracy_nb: {}".format(accuracy_nb))


In [61]:
#Train-test split
print("... Performing train test split")
X_train, X_test, y_train, y_test = model_selection.train_test_split(corpus,df[labels],
                                                                    test_size=0.25,random_state=42)

... Performing train test split


In [62]:
## Features extraction with word embedding
print("... Extracting features")
Xv_train, Xv_test, vectorizer = get_embeddings(X_train, X_test,
                                                          max_feature = 1000 , embedding_type= 'bow')

... Extracting features
